In [66]:
import numpy as np
from scipy.spatial import Delaunay

import visit_writer

import structured_3d_vtk as struc_3d

# Define functions

In [89]:
def prepare_idl_quantity(name, planes):
    r"""
    """
    measurements = struc_3d.read_idl(name)
    for plane in planes:
        measurements[plane] = struc_3d.average_duplicate_points(measurements[plane])
    return measurements

In [90]:
def combine_all_planes(measurements, planes):
    r"""
    """
    all_planes = dict(measurements[planes[0]])
    all_planes['z_out'] = planes[0]*np.ones(all_planes['x_out'].size)
 
    for plane in planes[1:]:
        for key in ['x_out', 'y_out']:
            all_planes[key] = np.concatenate((all_planes[key], 
                                              measurements[plane][key]))
        all_planes['z_out'] = np.concatenate((all_planes['z_out'],
                                              plane*np.ones(measurements[plane]['x_out'].size)))
        for key in ['std', 'a_out']:
            for time_point in xrange(measurements[plane]['delays'].size):
                all_planes[key][time_point] = np.concatenate((all_planes[key][time_point], 
                                                              measurements[plane][key][time_point]))
    return all_planes

In [91]:
def remove_points_outside_convex(measurements,
                                 x_min= -0.021, x_max=0.016,
                                 y_min=-0.016, y_max=0.0135,
                                 z_min=0.249, z_max=0.416):
    r"""
    """
    points = np.dstack((measurements['x_out'], 
                        measurements['y_out'], 
                        measurements['z_out']))[0]
    outside_convex_volume = np.where(np.logical_or.reduce((points[:, 0] < x_min, 
                                                           points[:, 0] > x_max, 
                                                           points[:, 1] < y_min,
                                                           points[:, 1] > y_max,
                                                           points[:, 2] < z_min,
                                                           points[:, 2] > z_max)))[0]
    for key in ['x_out', 'y_out', 'z_out']:
        measurements[key] = np.delete(measurements[key], 
                                      outside_convex_volume)
    for key in ['std', 'a_out']:
        for time_point in xrange(measurements['delays'].size):
            measurements[key][time_point] = np.delete(measurements[key][time_point], 
                                                      outside_convex_volume)
    return measurements

In [92]:
def prepare_for_unstructured_vtk(measurements, quantity_name):
    r"""
    """
    points = np.dstack((measurements['x_out'], 
                        measurements['y_out'], 
                        measurements['z_out']))[0]
    triangulation = Delaunay(points)
    points = tuple(points.ravel())
    connectivity = tuple([(visit_writer.tetrahedron, int(simplex[0]), 
                           int(simplex[1]), int(simplex[2]), int(simplex[3])) 
                           for simplex in triangulation.simplices])
    variables_all_time = []
    for time_point in xrange(measurements['delays'].size):
        variables = ((quantity_name, 1, 1, tuple(measurements['a_out'][time_point])),)
        variables_all_time.append(variables)
    return points, connectivity, variables_all_time

In [93]:
def write_all_time_unstructured(file_prefix, points, connectivity, variables_all_time):
    r"""
    """
    for time_point in xrange(len(variables_all_time)):
        path = file_prefix + str(time_point).zfill(4)
        visit_writer.WriteUnstructuredMesh(path, 1, points,
                                           connectivity, 
                                           variables_all_time[time_point])

In [107]:
def save_idl_quantity_to_unstructured_grids(idl_quantity_name,
                                            visit_quantity_name,
                                            date, 
                                            planes=[0.249, 0.302, 0.357, 0.416],
                                            file_name_descriptor='_all_planes_convex_unstructured_grid_',
                                            x_min= -0.021, x_max=0.016,
                                            y_min=-0.016, y_max=0.0135,
                                            z_min=0.249, z_max=0.416):
    file_prefix = '../output/' + date + '/' + visit_quantity_name + file_name_descriptor
    measurements = prepare_idl_quantity(idl_quantity_name, planes)
    all_planes = combine_all_planes(measurements, planes) 
    all_planes = remove_points_outside_convex(all_planes, 
                                              x_min=x_min, x_max=x_max,
                                              y_min=y_min, y_max=y_max,
                                              z_min=z_min, z_max=z_max)
    assert len(all_planes['x_out']) == len(all_planes['y_out']) == len(all_planes['z_out'])
    (points, 
     connectivity, 
     variables_all_time) = prepare_for_unstructured_vtk(all_planes, visit_quantity_name)
    assert len(points) == len(all_planes['x_out'])*3
    assert len(variables_all_time[0][0][3]) == len(all_planes['x_out'])
    write_all_time_unstructured(file_prefix, points, 
                                connectivity, variables_all_time)

# Write all idl quantities to files

In [108]:
save_idl_quantity_to_unstructured_grids('bx', 'B_x', '2016-07-26')

In [109]:
save_idl_quantity_to_unstructured_grids('by', 'B_y', '2016-07-26')

In [110]:
save_idl_quantity_to_unstructured_grids('bz', 'B_z', '2016-07-26')

In [111]:
save_idl_quantity_to_unstructured_grids('te', 'T_e', '2016-07-26', 
                                        x_min=-0.022, x_max =0.018,
                                        y_min=-0.021, y_max=0.0255, 
                                        z_min=0.249, z_max=0.416)

In [112]:
save_idl_quantity_to_unstructured_grids('n', 'n', '2016-07-26',
                                        x_min=-0.022, x_max =0.018,
                                        y_min=-0.021, y_max=0.0255, 
                                        z_min=0.249, z_max=0.416)